# 2048 Keras

In [1]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input, concatenate, BatchNormalization, Activation
from keras.optimizers import Adadelta
import numpy as np

BATCH_SIZE = 128
NUM_EPOCHS = 15

Using TensorFlow backend.


In [2]:
inputs = Input((4,4,16))

conv = inputs
FILTERS = 128
conv41 = Conv2D(filters=FILTERS, kernel_size=(4,1), kernel_initializer='he_uniform')(conv)
conv14 = Conv2D(filters=FILTERS, kernel_size=(1,4), kernel_initializer='he_uniform')(conv)
conv22 = Conv2D(filters=FILTERS, kernel_size=(2,2), kernel_initializer='he_uniform')(conv)
conv33 = Conv2D(filters=FILTERS, kernel_size=(3,3), kernel_initializer='he_uniform')(conv)
conv44 = Conv2D(filters=FILTERS, kernel_size=(4,4), kernel_initializer='he_uniform')(conv)

hidden = concatenate([Flatten()(conv41), Flatten()(conv14), Flatten()(conv22), Flatten()(conv33), Flatten()(conv44)])
x = BatchNormalization()(hidden)
x = Activation('relu')(hidden)

for width in [512,128]:
    x = Dense(width,kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
outputs = Dense(4,activation='softmax')(x)
model = Model(inputs, outputs)
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4, 4, 16)     0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 1, 4, 128)    8320        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 4, 1, 128)    8320        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 3, 3, 128)    8320        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (

In [3]:
OUT_SHAPE = (4,4)
CAND = 16
#map_table = {2**i : i for i in range(1,CAND)}
#map_table[0] = 0

def grid_one(arr):
    ret = np.zeros(shape=OUT_SHAPE+(CAND,),dtype=bool)  # shape = (4,4,16)
    for r in range(OUT_SHAPE[0]):
        for c in range(OUT_SHAPE[1]):
            ret[r,c,arr[r,c]] = 1
    return ret

In [4]:
import csv
data_raw = []
with open("train/train1M_1.csv") as f:
    for line in f:
        piece = eval(line)
        data_raw.append(piece)

In [5]:
len(data_raw)

1000000

In [45]:
import json 
data7 = []
with open("train7.json",'r') as f:
    data7 = json.load(f)

In [46]:
data7 = np.array(data7)
data = data7
data7.shape

(871430, 17)

In [7]:
data = np.array(data_raw)
data.shape

(1000000, 17)

In [8]:
x = np.array([ grid_one(piece[:-1].reshape(4,4)) for piece in data ])
y = keras.utils.to_categorical(data[:,-1], 4)

In [9]:
sep = 9000
x_train = x[:sep]
x_test = x[sep:]
y_train = y[:sep]
y_test = y[sep:]

In [10]:
x_test.shape

(991000, 4, 4, 16)

In [11]:
# train , validation_data=(x_test,y_test)
model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=1, verbose=1)

Epoch 1/1
9000/9000 [==============================] - 10s 1ms/step - loss: 1.4347 - acc: 0.3420


In [48]:
# evaluate
#score_train = model.evaluate(x_train,y_train,verbose=0)
#print('Training loss: %.4f, Training accuracy: %.2f%%' % (score_train[0],score_train[1]))
score_test = model.evaluate(x_test,y_test,verbose=0)
print('Testing loss: %.4f, Testing accuracy: %.2f' % (score_test[0],score_test[1]))

Testing loss: 1.6294, Testing accuracy: 0.58


In [19]:
model.save('model_k.h5')  # creates a HDF5 file 'my_model.h5'

In [10]:
# returns a compiled model
# identical to the previous one
model = keras.models.load_model('model_k.h5')

#### Q6:
Please print the training and testing accuracy.